# PyTorchの学習済みモデルを自由自在に書き換えたい

PyTorchには[PyTorch Image Models](https://github.com/rwightman/pytorch-image-models)など学習済モデルがたくさん公開されていて、これを転移学習に使うことも多いです。その際、学習済モデルを少し改造して試したい場合、どうすればいいのか。直接編集するわけではありませんが、同等の効果がある方法をご紹介します。

```
要点
1. 挿入するにはtorch.nn.Sequentialで置き換える
2. 削除するにはtorch.nn.Indentityで置き換える
```





まずはtorch.hubからお馴染みのresnet18を取得して、その構造を表示してみましょう。

In [1]:
import torch
model = torch.hub.load('rwightman/pytorch-image-models:master', 'resnet18', pretrained=True)
print(model)

Using cache found in /root/.cache/torch/hub/rwightman_pytorch-image-models_master


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

括弧で括られた部分がそのレイヤーの名前です。例えば(fc)となっている最終レイヤーにアクセスするには以下のようにします。

In [2]:
print(model.fc)

Linear(in_features=512, out_features=1000, bias=True)


ここで、出力の特徴数を変更するには以下です。

In [3]:
model.fc.out_features = 100
print(model.fc)

Linear(in_features=512, out_features=100, bias=True)


同様にレイヤーを置き換えるのは簡単です。

In [4]:
print('before:', model.act1)
model.act1 = torch.nn.LeakyReLU(inplace=True)
print('after:', model.act1)

before: ReLU(inplace=True)
after: LeakyReLU(negative_slope=0.01, inplace=True)


それではあるレイヤーを挿入したい場合はどうすればいいでしょうか。レイヤーを挿入するメソッドを見つけられなかったので、torch.nn.Sequentialを使って、以下のようにしました。以下はfc層の直前にDropoutを挿入する例です。

In [5]:
from torch.nn import Sequential, Dropout
model.fc = Sequential(Dropout(0.5), model.fc)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): LeakyReLU(negative_slope=0.01, inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tr

上手く挿入されました。

それではレイヤを削除したい場合はどうすればいいでしょうか。例えば最終層のクラス分類を外して、直前のレイヤーの出力を利用したいような場合です。この場合も、レイヤーを削除するメソッドが見つけられなかったので、以下のようにnn.Identityで置き換えます。

In [6]:
model.fc = torch.nn.Identity()
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): LeakyReLU(negative_slope=0.01, inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tr

これで一つ前の(global_pool)の出力をそのまま出力するようにできたので、クラス分類層を削除したのと同様になりました。

ところで、モデルの構造をよく見ると、(0)のように示されているものがあります。これはmodel.layer1.0などではアクセスできませんので、以下のようにします。

In [7]:
model.layer1[0]

BasicBlock(
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): ReLU(inplace=True)
)

無事にアクセスできました。これでどんな層も自由に編集できます。

以上です。